In [ ]:
from utility.classes.data_extraction import DataExtractor
from utility.classes.stability_analysis import StabilVolter, MeanFirstHittingTimes

import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from pathlib import Path
import pandas as pd

DATABASE = Path('../data/interim')

MARKET = 'UN'
CRITERION = 'percentage'
VALUE = 0.8

START_LEVEL = -0.1
END_LEVEL = -1.5
TAU_MAX = 1e6
NBINS = 1000

In [ ]:
def print_indicators_table(header, indicators):
    print(header)
    keys = list(indicators.keys())
    values = list(indicators.values())
    keys_width = max(len(key) for key in keys)
    values_width = max(len(str(value)) for value in values)
    table_width = keys_width + values_width + 5
    print(f"{'-' * table_width}")
    print(f"|{'Key':^{keys_width}}|{'Value':^{values_width}}|")
    print(f"{'-' * table_width}")
    for key, value in indicators.items():
        value_str = str(value) if not isinstance(value, tuple) else ', '.join(map(str, value))
        print(f"|{key:<{keys_width}}|{value_str:^{values_width}}|")
    print(f"{'-' * table_width}")

def analyze_and_plot_mfht(start_date, end_date, plot=True, visible=True):
    accountant = DataExtractor(
            start_date=start_date,
            end_date=end_date,
            criterion=CRITERION,
            criterion_value=VALUE,
            sigma_range=(1e-5, 1e5)
    )
    data = accountant.extract_data(DATABASE / f'{MARKET}.pickle')

    analyst = StabilVolter(
        start_level=START_LEVEL,
        end_level=END_LEVEL,
        tau_max=TAU_MAX)
    analysis_info = {
        'Market': MARKET,
        'Start date': accountant.start_date.strftime("%Y-%m-%d"),
        'Window length': int(accountant.window.length.days / 365.2425)
    }
    start_time = time()
    stabilvol = analyst.get_stabilvol(data, 'multi', **analysis_info)
    end_time = time()
    mfht = MeanFirstHittingTimes(stabilvol, nbins=NBINS, max_volatility=0.2)
    if plot:
        if visible:
            ax = mfht.plot(edit=True, indicators=False)
            ax.set_ylim(0, 55)
            ax.set_xlim(0, 0.2)
            plt.savefig(f'../visualization/dynamics/mfht_naked.png', dpi=300, transparent=False)
            plt.show()
            ax = mfht.plot(edit=True)
            ax.set_ylim(0, 55)
            ax.set_xlim(0, 0.2)
            plt.savefig(f'../visualization/dynamics/mfht.png', dpi=300, transparent=False)
            plt.show()
        ax = mfht.plot(invisible=True, edit=True)
        ax.set_ylim(0, 55)
        ax.set_xlim(0, 0.2)
        plt.savefig(f'../visualization/dynamics/mfht_indicators_0.png', dpi=300, transparent=True)
        plt.show()
    return mfht.indicators

In [ ]:
start_date = '1990-01-01'
end_date = '1995'

analyze_and_plot_mfht(start_date, end_date, visible=True)

In [ ]:
dates = pd.date_range(start='1990-01-01', end='2000-01-01', freq='4MS').to_list()
indicators = []
for start_date in tqdm(dates):
    end_date = start_date + pd.DateOffset(years=5)
    mfht_indicators = analyze_and_plot_mfht(start_date, end_date, plot=False, visible=False)
    indicators.append(mfht_indicators)

In [ ]:
indicators_df = pd.DataFrame(indicators)
indicators_df.index = dates
indicators_df.plot(subplots=True)
indicators_df.max()

In [ ]:
params = {"ytick.color" : "w", "xtick.color" : "w", "axes.labelcolor" : "w", "axes.edgecolor" : "w"}
plt.rcParams.update(params)
for start_date in tqdm(dates):
    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle('Mean First Hitting Times', color='white')
    # Set the figure background color to black
    ax.set_facecolor('black')
    ax.set_title(f'{NBINS} bins', color='white')
    index = pd.Timestamp(start_date)
    y = indicators_df.loc[start_date, 'Max']
    x = indicators_df.loc[start_date, 'Peak']
    fwhm = indicators_df.loc[start_date, 'FWHM']
    # Max Point
    ax.axhline(y=y, ls='--', c='r')
    ax.axvline(x=x, ls='--', c='r')
    ax.scatter(x=x, y=y, c='r')
    # Width
    left_range = x - fwhm / 2
    right_range = x + fwhm / 2
    ax.axvspan(left_range, right_range, color='r', alpha=0.2)
    # Set limits
    ax.set_ylim(0, 55)
    ax.set_xlim(0, 0.2)
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    # Save
    start_date_string = start_date.strftime("%Y_%m_%d")
    plt.savefig(f'../visualization/dynamics/indicators_evolution/mfht_indicators_{start_date_string}.png',
                dpi=300, facecolor='black')
    # Show
    # plt.show()
    plt.close()

In [ ]:
import os
from PIL import Image

# Create the frames
frames = []
path = "../visualization/dynamics/indicators_evolution"
for frame in os.listdir(path):
    new_frame = Image.open(path + "/" + frame)
    frames.append(new_frame)

# Save into a GIF file
frames[0].save("../visualization/dynamics/output/animation.gif", format='GIF', append_images=frames[1:], save_all=True, duration=240, loop=0)